In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# modifiy default model of crewAI
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

In [2]:
# ROE = Return on Equity -> company overview
# PER = Price Earning Ratio -> company overview
# FFO = 당기순이익 - 비영업이익 -> Net Income - OtherNonOperatingIncome    --->   비영업수익 : income_statement

# 배당률 = dividendpayout / net Income

In [3]:
from crewai_tools import tool
import requests


class Tools:
    @tool("Information of Traditional Indicator")
    def get_information_of_traditional_indicator():
        """
        Give information about traditional indicator with analysing company stock.
        """
        return """
        There are three Traditional Indicator you should check. 
        ROE (Return on Equity):
        ROE measures the efficiency and profitability of a company's equity investment. A higher ROE indicates more effective management and profitability, with a benchmark of 15% being desirable. Companies with high ROE are often well-managed and show potential for long-term growth.

        PER (Price Earning Ratio):
        PER assesses if a stock is undervalued or overvalued by comparing the stock price to its EPS. A PER below 1 suggests the stock may be undervalued, while a PER above 1 indicates possible overvaluation. Investing in stocks with a PER below 1 is generally seen as a wise choice.

        FFO (Funds From Operations):
        FFO reflects earnings from core operations by adjusting net income for non-operational gains and expenses. It's a more accurate measure of a company’s operational performance compared to net income. Companies with consistently growing FFO over the past five years are considered good investment opportunities.
        """
    
    @tool("Get ROE rate of company")
    def get_ROE(symbol):
        """
        A tool to get ROE of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        req = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")

        return req.json()["ReturnOnEquityTTM"]
    
    @tool("Get PER rate of company")
    def get_PER(symbol):
        """
        A tool to get PER of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        req = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")

        return req.json()["PERatio"]
    
    @tool("Get 5years FFO rate of company")
    def get_FFO(symbol):
        """
        A tool to get 5 years FFO of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        req = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")

        list_five_years_annual_report = req.json()['annualReports'][0:4]


        FFO = dict()

        for report in list_five_years_annual_report:
            date = report['fiscalDateEnding']
            ffo = float(report['netIncome']) - float(report['otherNonOperatingIncome'])

            FFO[date] = ffo

        return FFO  

    @tool("Get Dividend Rate of company")
    def get_diviend(symbol):
        
        """
        A tool to get Dividend Rate of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        cash_flow = requests.get(f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}").json()
        net_income = cash_flow['annualReports'][0]['netIncome']
        dividend_payout = cash_flow['annualReports'][0]['dividendPayout']

        dividend_rate = round(float(dividend_payout) / float(net_income), 2)

        return dividend_rate

    @tool("Get Recent News of company")
    def get_company_news(symbol):
        """
        A tool to get News of company.
        This data contain sentiment data but it doesn't uncertain. Use this data carefully.
        Also, this data is json format.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        news = requests.get(f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={alpha_vantage_api_key}")
        
        return news.json()

    @tool("Get Inflation rate")
    def get_inflation_rate():
        """
        A tool to get Inflation rate of US.
        This data is json format.
        """

        inflation = requests.get("https://www.alphavantage.co/query?function=INFLATION&apikey={alpha_vantage_api_key}")

        return inflation.json()
    
    @tool("Get Stock Price")
    def get_stock_price(symbol):
        """
        A tool to get stock price of company.
        This data is very big. You use carefully use this data.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        stock_price = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=IBM&apikey={alpha_vantage_api_key}")

        return stock_price.json()["Monthly Adjusted Time Series"]


In [4]:
from crewai import Task

class Tasks:
    
    def symbol_research(self, agent):
        return Task(
            description="Gather information of {company} stock symbol",
            expected_output=f"Your Final Answer MUST be a stock symbol",
            agent=agent,
        )  
    def indicator_research(self, agent):
        return Task(
            description="Gather information of Traditional Company Indicator and get value of Indicator. You should assess the {company} with provided information",
            expected_output=f"Your Final Answer Must be three value of traditional indicator. And your information of Indicator must be shared with other agents.",
            agent=agent,
        )    
    def dividend_rate_analysis(self, agent):
        return Task(
            description="Get value of dividend rate of {company}. And judge whether {company} is good to invest",
            expected_output=f"Your Final Answer Must be company's dividend rate and judgement whether this company is good to invest.",
            agent=agent,
        )   
    def news_analysis(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance",
            agent=agent,
        )   
    def market_share_analysis(self, agent):
        return Task(
            description="Gather and analyze the market share of {company}. You can use whatever you want",
            expected_output=f"Your final answer Must be detail summary about market share of company",
            agent=agent,
        ) 
    def stock_price_analyst(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions",
            agent=agent,
        )  
    def hedge_fund_manager(self, agent, context):
        return Task(
            description="Based on the response of other agents, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports",
            agent=agent,
            context=context,
            output_file="Result.md"
        )

In [5]:
from crewai import Agent
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


class Agents:
    def symbol_researcher(self):
        return Agent(
            role="Symbol_Researcher",
            goal="Search and find company's symbol.",
            backstory="You're skilled in finding company's stock symbol. You have to find company's stock symbol with company's name",
            verbose=True,
            tool = [
                SerperDevTool(),
                ScrapeWebsiteTool()
            ],
        )  
    def traditional_indicator_analyst(self):
        return Agent(
            role="Traditional Indicator Analyst",
            goal="Analyses the three traditional indicator and provides insight of this company.",
            backstory="An expert in indicator analysis, you have to get information of tranditional indicator and analysis indicator based on that information. You give advice to important customer.",
            verbose=True,
            tool = [
                Tools.get_information_of_traditional_indicator,
                Tools.get_ROE,
                Tools.get_PER,
                Tools.get_FFO,
            ]
        )
    def dividend_rate_analyst(self):
        return Agent(
            role="Dividend Rate Analyst",
            goal="Analyses the dividend rate of company and provides insight of this company.",
            backstory="You are an expert in judging Dividend Rate. Get Dividend Rate of company and check out Dividend Rate is higher than 40%. If Dividend Rate is higher than 40%, this company is good to invest.",
            verbose=True,
            tool = [
                Tools.get_diviend,
            ]
        )
    def news_analyst(self):
        return Agent(
            role="News Analyst",
            goal="Analyses the news of given company. Get the sentiment of each news and analysis if this company is good to invest.",
            backstory="You are an Analyst with news. You analysis each of company news and get sentiment. You should give insight whether it is good to invest this company. Also, return the recent important news of company. You are giving a advice to important customer.",
            verbose=True,
            tool = [
                Tools.get_company_news,
            ]  
        )
    def market_share_analyst(self):
        return Agent(
            role="Market Share Analyst",
            goal="Analyses the Market Share of company. You can use any tool to get Market Share of Company. If you cannot find about Market Share, just say 'I don't Know'. Do not MAKEUP your owns.",
            backstory="You are an expert on analysis market share of company. You can use your tools to get information of company Market Share. If you don't know, just I don't know. DO NOT MAKEUP.",
            verbose=True,
            tool = [
                SerperDevTool(),
                ScrapeWebsiteTool(),
            ]
        )
    def stock_price_analyst(self):
        return Agent(
            role="Stock Price Analyst",
            goal="Analyses the three traditional indicator and provides insight of this company.",
            backstory="You are an expert in Price Analyses. You should judge company stock price with following three statement. First, company's stock price increasement should be higher than market inflation. Second, stock price should not be decrease 15% in recent 1 years. All of this statement is satisfy, this company is good to invest. You should analysis stock price and give response to customer if this company is good to invest.",
            verbose=True,
            tool = [
                Tools.get_inflation_rate,
                Tools.get_stock_price,
            ]
        )
    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from Traditional Indicator Analyst, Dividend Rate Analyst, News Analyst, Market Share Analyst, Stock Price Analyst. Also, You MUST CONSIDER importance of each data.",
            backstory="You are the head fund manager. You need to decide whether to buy this company's stock by listening to the opinions of the Traditional Indicator Analyst, Dividend Rate Analyst, News Analyst, Market Share Analyst, and Stock Price Analyst. The importance of each piece of information varies. The importance decreases from left to right, meaning the priority of information is as follows: traditional indicator > dividend rate > news > market share > stock price. Additionally, you are currently dealing with a very important client. ",
            verbose=True,
        )
    

In [6]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
task = Tasks()

symbol_researcher = agents.symbol_researcher()
traditional_indicator_analyst = agents.traditional_indicator_analyst()
dividend_rate_analyst = agents.dividend_rate_analyst()
news_analyst = agents.news_analyst()
market_share_analyst = agents.market_share_analyst()
stock_price_analyst = agents.stock_price_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

symbol_research__Task = task.symbol_research(symbol_researcher)
indicator_research__Task = task.indicator_research(traditional_indicator_analyst)
dividend_rate_analysis__Task = task.dividend_rate_analysis(dividend_rate_analyst)
news_analysis__Task = task.news_analysis(news_analyst)
market_share_analysis__Task = task.market_share_analysis(market_share_analyst)
stock_price_analyst__Task = task.stock_price_analyst(stock_price_analyst)
hedge_fund_manager__Task = task.hedge_fund_manager(hedge_fund_manager, [symbol_research__Task, indicator_research__Task, dividend_rate_analysis__Task, news_analysis__Task, market_share_analysis__Task, stock_price_analyst__Task])

crew = Crew(
    agents=[symbol_researcher, traditional_indicator_analyst, dividend_rate_analyst, news_analyst, market_share_analyst, stock_price_analyst, hedge_fund_manager],
    tasks=[symbol_research__Task, indicator_research__Task, dividend_rate_analysis__Task, news_analysis__Task, market_share_analysis__Task, stock_price_analyst__Task, hedge_fund_manager__Task],
    verbose=2,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(inputs=dict(company="Nivida"))







 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather information of Nivida stock symbol


> Entering new CrewAgentExecutor chain...
To gather information on the NVIDIA stock symbol, I need to delegate the task to the appropriate co-worker who specializes in stock symbol research. The most suitable choice would be the Symbol Researcher, as they focus on researching stock symbols and their relevant information.

Action: Delegate work to co-worker
Action Input: {"co-worker": "Symbol_Researcher", "task": "Gather comprehensive information about the NVIDIA stock symbol, including its current market price, historical performance, trading volume, and any relevant news or updates that may impact its market behavior. Ensure to check for any recent changes or critical events related to NVIDIA that could influence investor sentiment.", "context": "This task requires an in-depth analysis of NVIDIA's stock symbol to provide a detailed understanding of its market position and outlook.